In [1]:
import tensorflow as tf

trainingSize = 10
featureSize = 28*28

x = tf.placeholder(tf.float32, [None, featureSize])
W = tf.Variable(tf.zeros([featureSize, trainingSize]))
b = tf.Variable(tf.zeros([trainingSize]))

In [2]:
y = tf.nn.softmax(tf.matmul(x,W) + b)

y_ = tf.placeholder(tf.float32, [None, trainingSize])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [3]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [6]:
testing = mnist.test.images
testing_labels = mnist.test.labels

print (sess.run(accuracy, feed_dict={x: testing, y_: testing_labels}))
feed_dict = {x: testing}
classification = sess.run(y, feed_dict)
print classification.shape
print testing.shape
for i in range(10):
    test = classification[i]
    label = testing_labels[i]
    print str(test.argmax()) + " " + str(label.argmax())

0.9188
(10000, 10)
(10000, 784)
7 7
2 2
1 1
0 0
4 4
1 1
4 4
9 9
6 5
9 9


In [7]:
def variable_summaries(var, name):
    with tf.name_scope("summaries"):
        mean = tf.reduce_mean(var)
        tf.scalar_summary('mean/' + name, mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_sum(tf.square(var - mean)))
        tf.scalar_summary('sttdev/' + name, stddev)
        tf.scalar_summary('max/' + name, tf.reduce_max(var))
        tf.scalar_summary('min/' + name, tf.reduce_min(var))
        tf.histogram_summary(name, var)

def nn_layer(input_tensor, input_dim, output_dim, layer_name):
    """Reusable code for making a simple neural net layer.

    It does a matrix multiply, bias add, and then uses relu to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read, and
    adds a number of summary ops.
    """
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
        # This Variable will hold the state of the weights for the layer
        with tf.name_scope("weights"):
            weights = W([input_dim, output_dim])
            variable_summaries(weights, layer_name + '/weights')
        with tf.name_scope("biases"):
            biases = b([output_dim])
            variable_summaries(biases, layer_name + '/biases')
        with tf.name_scope('Wx_plus_b'):
            activations = tf.matmul(input_tensor, weights) + biases
            tf.histogram_summary(layer_name + '/activations', activations)
        relu = tf.nn.relu(activations, 'relu')
        tf.histogram_summary(layer_name + '/activations_relu', relu)
        return tf.nn.dropout(relu, keep_prob)

layer1 = nn_layer(x, 784, 50, 'layer1')
layer2 = nn_layer(layer1, 50, 10, 'layer2')
y = tf.nn.softmax(layer2, 'predictions')


with tf.name_scope('cross_entropy'):
    diff = y_ * tf.log(y)
    with tf.name_scope('total'):
        cross_entropy = -tf.reduce_sum(diff)
    with tf.name_scope('normalized'):
        normalized_cross_entropy = -tf.reduce_mean(diff)
    tf.scalar_summary('cross entropy', normalized_cross_entropy)

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(
            FLAGS.learning_rate).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.scalar_summary('accuracy', accuracy)

# Merge all the summaries and write them out to /tmp/mnist_logs (by default)
merged = tf.merge_all_summaries()
train_writer = tf.train.SummaryWriter(FLAGS.summaries_dir + '/train', sess.graph)
test_writer = tf.train.SummaryWriter(FLAGS.summaries_dir + '/test')
tf.initialize_all_variables().run()

TypeError: 'Variable' object is not callable